In [24]:
# !pip install torch==1.4.0+cpu torchvision==0.5.0+cpu -f https://download.pytorch.org/whl/torch_stable.html

In [25]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torchvision
import torch.utils.data as Data

In [26]:
BATCH_SIZE = 64
#学习率，学习率一般为0.01，0.1等等较小的数，为了在梯度下降求解时避免错过最优解
LR = 0.001
EPOCH = 1

In [27]:
# !pip3 install ipywidgets --user

In [28]:
train_data = torchvision.datasets.MNIST(root='.\data', 
                                        train=True, 
                                        transform=torchvision.transforms.ToTensor(),  
                                        download=True)

Using downloaded and verified file: .\data\MNIST\raw\train-images-idx3-ubyte.gz
Extracting .\data\MNIST\raw\train-images-idx3-ubyte.gz to .\data\MNIST\raw


0it [1:31:42, ?it/s]
0it [25:49, ?it/s]
0it [04:16, ?it/s]

Using downloaded and verified file: .\data\MNIST\raw\train-labels-idx1-ubyte.gz
Extracting .\data\MNIST\raw\train-labels-idx1-ubyte.gz to .\data\MNIST\raw
Using downloaded and verified file: .\data\MNIST\raw\t10k-images-idx3-ubyte.gz
Extracting .\data\MNIST\raw\t10k-images-idx3-ubyte.gz to .\data\MNIST\raw
Using downloaded and verified file: .\data\MNIST\raw\t10k-labels-idx1-ubyte.gz
Extracting .\data\MNIST\raw\t10k-labels-idx1-ubyte.gz to .\data\MNIST\raw
Processing...
Done!


In [29]:
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2 )
#每个batch_size的shape为[64, 1, 28, 28]

In [30]:
#测试集操作和上面注释一样
test_data = torchvision.datasets.MNIST(
    root='.\data',
    train = False,
)

In [31]:
test_x = Variable(torch.unsqueeze(test_data.test_data, dim=1), volatile=True).type(torch.FloatTensor)[:2000]/255.0
#标签取前2000
test_y = test_data.test_labels[:2000]

c:\Users\33398\.conda\envs\Aliyun\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
c:\Users\33398\.conda\envs\Aliyun\lib\site-packages\ipykernel_launcher.py:1: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """Entry point for launching an IPython kernel.
c:\Users\33398\.conda\envs\Aliyun\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [32]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #前面都是规定结构
        #第一个卷积层，这里使用快速搭建发搭建网络
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,#灰度图，channel为一
                out_channels=16,#输出channels自己设定
                kernel_size=3,#卷积核大小
                stride=1,#步长
                padding=1#padding=（kernel_size-stride）/2   往下取整
            ),
            nn.ReLU(),#激活函数，线性转意识到非线性空间
            nn.MaxPool2d(kernel_size=2)#池化操作，降维，取其2x2窗口最大值代表此窗口，因此宽、高减半，channel不变
        )
        #此时shape为[16, 14, 14]
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding=1
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        #此时shape为[32, 7, 7]
        #定义全连接层，十分类，并且全连接接受两个参数，因此为[32*7*7, 10]
        self.prediction = nn.Linear(32*7*7, 10)
        #前向传播过程
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.prediction(x)
        return output

In [33]:
cnn = CNN()

#大数据常用Adam优化器，参数需要model的参数，以及学习率
optimizer = torch.optim.Adam(cnn.parameters(), LR)
#定义损失函数，交叉熵
loss_func = nn.CrossEntropyLoss()

In [34]:
#训练阶段
for epoch in range(EPOCH):
    #step,代表现在第几个batch_size
    #batch_x 训练集的图像
    #batch_y 训练集的标签
    for step, (batch_x, batch_y) in enumerate(train_loader):
        #model只接受Variable的数据，因此需要转化
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)
        #将b_x输入到model得到返回值
        output = cnn(b_x)
        #计算误差
        loss = loss_func(output, b_y)
        #将梯度变为0
        optimizer.zero_grad()
        #反向传播
        loss.backward()
        #优化参数
        optimizer.step()
        #打印操作，用测试集检验是否预测准确
        if step%50 == 0:
            test_output = cnn(test_x)
            #squeeze将维度值为1的除去，例如[64, 1, 28, 28]，变为[64, 28, 28]
            pre_y = torch.max(test_output, 1)[1].data.squeeze()
            #总预测对的数除总数就是对的概率
            accuracy = float((pre_y == test_y).sum()) / float(test_y.size(0))
            print("epoch:", epoch,  "|test accuracy：%.4f" %accuracy)

epoch: 0 |test accuracy：0.1490
epoch: 0 |test accuracy：0.8415
epoch: 0 |test accuracy：0.8905
epoch: 0 |test accuracy：0.9155
epoch: 0 |test accuracy：0.9255
epoch: 0 |test accuracy：0.9335
epoch: 0 |test accuracy：0.9465
epoch: 0 |test accuracy：0.9460
epoch: 0 |test accuracy：0.9505
epoch: 0 |test accuracy：0.9535
epoch: 0 |test accuracy：0.9545
epoch: 0 |test accuracy：0.9630
epoch: 0 |test accuracy：0.9645
epoch: 0 |test accuracy：0.9660
epoch: 0 |test accuracy：0.9670
epoch: 0 |test accuracy：0.9710
epoch: 0 |test accuracy：0.9675
epoch: 0 |test accuracy：0.9730
epoch: 0 |test accuracy：0.9715


In [35]:
print(output)

tensor([[ -6.7203, -17.8591,   0.8596,  -4.9627,  -8.4965,  -0.9482,  -6.5672,
          -3.4619,  -2.5276,  -1.6643],
        [ -3.4920,   7.6434,  -3.6701,  -3.8393,   0.3460,  -4.8533,  -2.6877,
          -2.5776,  -1.0976,  -4.4321],
        [ -2.9913,   7.5144,  -2.1815,  -3.9086,  -1.0041,  -4.7809,  -2.9428,
          -2.8509,  -1.3397,  -4.8424],
        [ -4.7841, -14.9725,  -4.0071,  -5.3980, -14.6892,  -3.2255,  -5.5932,
          -9.1255,  -0.0449,  -4.9689],
        [-14.1664,  -6.5079,  -8.0963,  -4.5434,   8.9957,  -4.2064,  -7.1382,
          -5.3441,   0.4848,  -0.9329],
        [ 12.9074, -16.8930,  -3.6951,  -9.8010, -16.5737,  -2.6237,   1.5323,
         -11.9687,  -2.6938,  -4.3762],
        [ -3.9486,   5.7839,  -0.5935,  -2.7641,  -2.2659,  -6.7892,  -5.8832,
          -1.7627,  -0.6090,  -2.7270],
        [ -6.3408,  -4.0967,  -3.2013,  -1.8224,  -5.2719,  -2.6525, -17.6906,
           7.5299,  -5.0006,   3.3216],
        [-11.7372,  -7.9817,  -8.2474,  -7.7006,